In [2]:
from bs4 import BeautifulSoup
import configparser
import requests
import pandas as pd
import pickle
from constants_funcs import get_session, int_or_float_or_str
import json
with open('constants.json', 'r') as d:
    constants = json.load(d)

In [34]:
sess = get_session()
sess

using existing login session


In [4]:
constants

{'LLHEADER': 'https://www.learnedleague.com',
 'LOGINFILE': 'https://www.learnedleague.com/ucp.php?mode=login',
 'USER_DATA': 'https://www.learnedleague.com/profiles/previous.php?%s',
 'QHIST': 'https://www.learnedleague.com/profiles/qhist.php?%s',
 'MATCH_DATA': 'https://www.learnedleague.com/match.php?%s',
 'ONEDAYS': 'https://www.learnedleague.com/oneday',
 'STANDINGS': '/standings.php?',
 'EXP_STANDINGS': '/standings_ex.php?',
 'ACTIVE_PLAYERS': '/backend-search.php?term=',
 'LLSTANDINGS': 'https://www.learnedleague.com/standings.php?',
 'ARUNDLE': 'https://www.learnedleague.com/standings.php?%d&A_%s',
 'INPUTDATA': 'logindata.ini',
 'TOTAL_MATCHES_PER_SEASON': 25}

In [24]:
all_questions = pd.read_csv('FULL_QUESTION_HISTORY.csv', sep=',', header=0, index_col=0)
idxs = all_questions.index.to_list()

In [25]:
all_questions['MCW'] = ''
all_questions['MCW_pct'] = None
all_questions

,season,matchday,qnum,question,answer,category,A_correct,B_correct,C_correct,D_correct,E_correct,R_correct,tot_correct,forfeit,defense,MCW,MCW_pct
59-1-1,59,1,1,The formal agreement known today as the Tripar...,"GERMANY, ITALY, JAPAN",WORLD HIST,88,82,71,59,35,50,60,4.0,0.9,,None
59-1-2,59,1,2,The domicile pictured here features prominentl...,AMERICAN GOTHIC,ART,90,85,82,77,50,75,74,4.0,1.4,,None
59-1-3,59,1,3,"While Fănică Luca, Dalila Cernătescu, and Simi...",GHEORGHE ZAMFIR,CLASS MUSIC,74,56,49,31,19,30,38,4.0,2.0,,None
59-1-4,59,1,4,"Name the man, a pivotal figure in the history ...",CURT FLOOD,GAMES/SPORT,59,52,39,27,15,22,31,4.0,1.6,,None
59-1-5,59,1,5,"Schlemiel! Schlimazel! This dish is a thick, h...",HASENPFEFFER,FOOD/DRINK,89,73,63,47,26,41,51,4.0,1.4,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93-25-2,93,25,2,"Identify the Dravidian language, particularly ...",TELUGU,LANGUAGE,27,18,11,8,5,11,11,3.0,2.2,,None
93-25-3,93,25,3,"The Statue of Liberty National Monument, locat...",ELLIS ISLAND,GEOGRAPHY,94,93,90,86,79,82,87,3.0,0.9,,None
93-25-4,93,25,4,Four fictional television-cartoon heroes fell ...,"MICHELANGELO, LEONARDO, DONATELLO, RAPHAEL",TELEVISION,96,95,94,92,86,90,91,3.0,0.7,,None
93-25-5,93,25,5,A cognate of an Old English word for a flexibl...,THONG,LIFESTYLE,47,44,39,33,27,32,36,3.0,1.6,,None


In [30]:
import re
def get_mcw(qid: str):
    season, md, qnum = qid.split('-')
    if int(season) < 56: # MCW not available for LL55 and earlier.
        return '', None
    url = constants['LLHEADER']+f'/question.php?{season}&{md}&{qnum}'
    ids_html = sess.get(url)
    html_text = ids_html.text
    soup = BeautifulSoup(html_text, 'html.parser')
    txt_arr = re.split('[\n\t]+', soup.find('div', attrs={'class': 'indivqContent'}).find('div').find_all('div')[4].text)
    mcw_idx = txt_arr.index('Most Common Wrong Answer')
    mcw = txt_arr[mcw_idx + 1]
    if mcw.find('No answer') > -1:
        return '', None
    text = ' '.join(mcw.split()[:-1])
    pct = int(mcw.split()[-1][1:-2])
    return text, pct
sample = '57-5-6'
mcw, pct = get_mcw(sample)
all_questions['MCW'][sample] = mcw
all_questions['MCW_pct'][sample] = pct
all_questions.loc[sample]

<ipython-input-30-b5eb6f4e2255>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_questions['MCW'][sample] = mcw
<ipython-input-30-b5eb6f4e2255>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_questions['MCW_pct'][sample] = pct


season                                                        57
matchday                                                       5
qnum                                                           6
question       "Because it is my name! Because I cannot have ...
answer                                              THE CRUCIBLE
category                                                 THEATRE
A_correct                                                     90
B_correct                                                     84
C_correct                                                     72
D_correct                                                     58
E_correct                                                     42
R_correct                                                     59
tot_correct                                                   63
forfeit                                                      4.7
defense                                                      1.8
MCW                      

In [38]:
# all_questions.loc['56-5-1']
for num, idx in enumerate(idxs):
    if all_questions['MCW_pct'][idx] is None and int(idx.split('-')[0]) >= 76:
        mcw, pct = get_mcw(idx)
        all_questions['MCW'][idx] = mcw
        all_questions['MCW_pct'][idx] = pct
    if num % 1000 == 0:
        print(num)
all_questions

0


<ipython-input-38-ab2346c4ed76>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_questions['MCW'][idx] = mcw
<ipython-input-38-ab2346c4ed76>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_questions['MCW_pct'][idx] = pct


1000
2000
3000
4000
5000
6000


,season,matchday,qnum,question,answer,category,A_correct,B_correct,C_correct,D_correct,E_correct,R_correct,tot_correct,forfeit,defense,MCW,MCW_pct
59-1-1,59,1,1,The formal agreement known today as the Tripar...,"GERMANY, ITALY, JAPAN",WORLD HIST,88,82,71,59,35,50,60,4.0,0.9,,None
59-1-2,59,1,2,The domicile pictured here features prominentl...,AMERICAN GOTHIC,ART,90,85,82,77,50,75,74,4.0,1.4,,None
59-1-3,59,1,3,"While Fănică Luca, Dalila Cernătescu, and Simi...",GHEORGHE ZAMFIR,CLASS MUSIC,74,56,49,31,19,30,38,4.0,2.0,YANNI,23
59-1-4,59,1,4,"Name the man, a pivotal figure in the history ...",CURT FLOOD,GAMES/SPORT,59,52,39,27,15,22,31,4.0,1.6,PETE ROSE,11
59-1-5,59,1,5,"Schlemiel! Schlimazel! This dish is a thick, h...",HASENPFEFFER,FOOD/DRINK,89,73,63,47,26,41,51,4.0,1.4,GOULASH,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93-25-2,93,25,2,"Identify the Dravidian language, particularly ...",TELUGU,LANGUAGE,27,18,11,8,5,11,11,3.0,2.2,,None
93-25-3,93,25,3,"The Statue of Liberty National Monument, locat...",ELLIS ISLAND,GEOGRAPHY,94,93,90,86,79,82,87,3.0,0.9,,None
93-25-4,93,25,4,Four fictional television-cartoon heroes fell ...,"MICHELANGELO, LEONARDO, DONATELLO, RAPHAEL",TELEVISION,96,95,94,92,86,90,91,3.0,0.7,,None
93-25-5,93,25,5,A cognate of an Old English word for a flexibl...,THONG,LIFESTYLE,47,44,39,33,27,32,36,3.0,1.6,,None


In [ ]:
all_questions.to_csv('FULL_QUESTION_HISTORY.csv')